# Install protinfer 

```
git clone https://github.com/google-research/proteinfer
cd ~/proteinfer
pip3 install -r requirements.txt
python3 install_models.py
```

## Train the model
Unfortunatly insufficient data is provided to train the model.
```
python train.py --data_base_path=./testdata/ \
--label_vocab_path=./data/vocabs/EC.tsv \
--hparams_set=small_test_model \
--output_dir=trained_baseline1
```
Next run the following:

```
python3 proteinfer.py -i /disk1/ariane/pycharm/CARE/CLEAN/app/data/price.fasta -o ~/hemoglobin_predictions.tsv
```


In [20]:
from sciutil import SciUtil
u = SciUtil()

base_dir = '/disk1/ariane/pycharm/CARE/'

# Input a ranked ordered list of EC numbers from best to worst.

# I'll 
    
def compute_accuracy_baseline1(predicted_ecs, true_ecs):
    pred_level1 = np.zeros(len(predicted_ecs))
    pred_level2 = np.zeros(len(predicted_ecs))
    pred_level3 = np.zeros(len(predicted_ecs))
    pred_level4 = np.zeros(len(predicted_ecs))

    # Basically given we have a highly multiclass problem doing precision recall probably doesn't make sense
    for i, predicted_ec in enumerate(predicted_ecs):
        true_ec = true_ecs[i]
        # First check level 1
        pred1 = predicted_ec.split('.')[0]
        true_ecs1 = [ec.split('.')[0] for ec in true_ec.split(';')]
        if pred1 in true_ecs1:
            pred_level1[i] = 1

        # Do the same for each other level
        pred2 = predicted_ec.split('.')[1]
        true_ecs2 = [ec.split('.')[1] for ec in true_ec.split(';')]
        if pred2 in true_ecs2:
            # Check also that pred1 was correct
            if pred_level1[i] == 1:
                pred_level2[i] = 1
        
        # Do the same for each other level
        pred3 = predicted_ec.split('.')[2]
        true_ecs3 = [ec.split('.')[2] for ec in true_ec.split(';')]
        if pred3 in true_ecs3:
            # Check previous levels were correct
            if pred_level1[i] == 1 and pred_level2[i] == 1:
                pred_level3[i] = 1
        
        # Do the same for each other level
        pred4 = predicted_ec.split('.')[3]
        true_ecs4 = [ec.split('.')[3] for ec in true_ec.split(';')]
        if pred4 in true_ecs4:
            if pred_level1[i] == 1 and pred_level2[i] == 1 and pred_level3[i] == 1:
                pred_level4[i] = 1
    # Print out the accuracy
    u.dp(['Acc level 1:', round(np.mean(pred_level1)*100, 2), 
          '\nAcc level 2:', round(np.mean(pred_level2)*100, 2), 
          '\nAcc level 3:', round(np.mean(pred_level3)*100, 2), 
          '\nAcc level 4:', round(np.mean(pred_level4)*100, 2)])
    
    return np.mean(pred_level1), np.mean(pred_level2), np.mean(pred_level3), np.mean(pred_level4)


def make_default_training_fasta():
    swissprot = pd.read_csv(f'{base_dir}processed_data/protein2EC.csv')
    train_indices = np.loadtxt(f'{base_dir}splits/task1/protein2EC_train_indices.txt', dtype=int)
    swissprot_train = swissprot.iloc[train_indices]
    entries = swissprot_train['Entry'].values
    sequences = swissprot_train['Sequence'].values
    #save sequences to fasta
    with open(f'{base_dir}protein2EC_train.fasta', 'w') as f:
        for entry, sequence in zip(entries, sequences):
            f.write('>{}\n{}\n'.format(entry, sequence))
    u.dp(['Default training dataset built using: ', f'{base_dir}processed_data/protein2EC.csv', 
          '\nIncides:', f'{base_dir}splits/task1/protein2EC_train_indices.txt', 
          '\nOutput:', f'{base_dir}protein2EC_train.fasta'])

def make_default_price_fasta():
    df = pd.read_csv(f'{base_dir}splits/task1/price_protein_test.csv', sep='\t')
    #write seqs to fasta format
    with open(f'{base_dir}splits/task1/price_protein_test.fasta', 'w') as f:
        for i, (entry, seq) in enumerate(zip(entries, seqs)):
            f.write('>{}\n{}\n'.format(entry, seq))
    u.dp(['Default price dataset built using: ', f'{base_dir}splits/task1/price_protein_test.csv', 
      '\nOutput:', f'{base_dir}splits/task1/price_protein_test.fasta'])

def make_fastas():
    filenames = [f'{base_dir}splits/task1/30-50_protein_test.csv', 
                 f'{base_dir}splits/task1/30_protein_test.csv', 
                 f'{base_dir}splits/task1/50-70_protein_test.csv',
                 f'{base_dir}splits/task1/70-90_protein_test.csv',
                 f'{base_dir}splits/task1/promiscuous_protein_test.csv',
                 f'{base_dir}splits/task1/protein_train.csv',
                 f'{base_dir}splits/task1/price_protein_test.csv']

    for filename in filenames:
        with open(filename.replace('.csv', '.fasta'), 'w') as f:
            df = pd.read_csv(filename)
            for entry, seq in df[['Entry', 'Sequence']].values:
                f.write('>{}\n{}\n'.format(entry, seq))
                     
def get_uniprot2ec():
    swissprot = pd.read_csv(f'{base_dir}processed_data/protein2EC.csv')
    id2ec = swissprot.set_index('Entry')['EC number'].to_dict()
    return id2ec

def get_price2ec():
    df = pd.read_csv(f'{base_dir}splits/task1/price_protein_test.csv')
    id2ec = df.set_index('Entry')['EC number'].to_dict()
    return id2ec
    
def get_default_training_fasta_path():
    return f'{base_dir}splits/task1/protein_train.fasta'
    
def get_default_price_fasta_path():
    return f'{base_dir}splits/task1/price_protein_test.fasta'

def get_validation30():
    return f'{base_dir}splits/task1/30_protein_test.fasta'

def get_validation50():
    return f'{base_dir}splits/task1/30-50_protein_test.fasta'
    
def get_validation70():
    return f'{base_dir}splits/task1/50-70_protein_test.fasta'

def get_validation90():
    return f'{base_dir}splits/task1/70-90_protein_test.fasta'

def get_promisc():
    return f'{base_dir}splits/task1/promiscuous_protein_test.fasta'

## Run protinfer

Note protinfer was not re-trained so may be slightly biased.

Protinfer needs to be installed as per the guidelines and then the following command can be run from the protinfer folder:

In [22]:
import os

filenames = [f'{base_dir}splits/task1/30-50_protein_test.csv', 
             f'{base_dir}splits/task1/30_protein_test.csv', 
             f'{base_dir}splits/task1/50-70_protein_test.csv',
             f'{base_dir}splits/task1/70-90_protein_test.csv',
             f'{base_dir}splits/task1/promiscuous_protein_test.csv',
             f'{base_dir}splits/task1/protein_train.csv',
             f'{base_dir}splits/task1/price_protein_test.csv']
for f in filenames:
    print(f'python3 proteinfer.py -i {f.replace(".csv", ".fasta")} -o /disk1/ariane/pycharm/CARE/output/protinfer/{f.split("/")[-1].replace(".csv", ".tsv")}')

python3 proteinfer.py -i /disk1/ariane/pycharm/CARE/splits/task1/30-50_protein_test.fasta -o /disk1/ariane/pycharm/CARE/output/protinfer/30-50_protein_test.tsv
python3 proteinfer.py -i /disk1/ariane/pycharm/CARE/splits/task1/30_protein_test.fasta -o /disk1/ariane/pycharm/CARE/output/protinfer/30_protein_test.tsv
python3 proteinfer.py -i /disk1/ariane/pycharm/CARE/splits/task1/50-70_protein_test.fasta -o /disk1/ariane/pycharm/CARE/output/protinfer/50-70_protein_test.tsv
python3 proteinfer.py -i /disk1/ariane/pycharm/CARE/splits/task1/70-90_protein_test.fasta -o /disk1/ariane/pycharm/CARE/output/protinfer/70-90_protein_test.tsv
python3 proteinfer.py -i /disk1/ariane/pycharm/CARE/splits/task1/promiscuous_protein_test.fasta -o /disk1/ariane/pycharm/CARE/output/protinfer/promiscuous_protein_test.tsv
python3 proteinfer.py -i /disk1/ariane/pycharm/CARE/splits/task1/protein_train.fasta -o /disk1/ariane/pycharm/CARE/output/protinfer/protein_train.tsv
python3 proteinfer.py -i /disk1/ariane/pycha

## Compute accuracy for the prediction vs the true values for each level

In [29]:
filenames = [f'{base_dir}splits/task1/30-50_protein_test.csv', 
             f'{base_dir}splits/task1/30_protein_test.csv', 
             f'{base_dir}splits/task1/50-70_protein_test.csv',
             f'{base_dir}splits/task1/70-90_protein_test.csv',
             f'{base_dir}splits/task1/promiscuous_protein_test.csv',
             f'{base_dir}splits/task1/price_protein_test.csv']

for f in filenames:
    u.dp(['Test set:', f.split('/')[-1]])
    results = pd.read_csv(f'/disk1/ariane/pycharm/CARE/output/protinfer/{f.split("/")[-1].replace(".csv", ".tsv")}', sep='\t')
    results['predicted_ecs'] = [ec.split(':')[1] if 'EC:' in ec else 'None' for ec in results['predicted_label'].values]
    # Remove ones without a predicted ec
    results = results[results['predicted_ecs'] != 'None']
    
    # Add in the actual ec
    if 'price' in f:
        results['true_ecs'] = results['sequence_name'].map(get_price2ec())
    else:
        results['true_ecs'] = results['sequence_name'].map(get_uniprot2ec())
        
    results = results.drop_duplicates(subset='sequence_name', keep='last')
    compute_accuracy_baseline1(results['predicted_ecs'].values, results['true_ecs'].values)
    

--------------------------------------------------------------------------------
                       Test set:	30-50_protein_test.csv	                        
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Acc level 1:	96.15	
Acc level 2:	92.86	
Acc level 3:	90.11	
Acc level 4:	84.07	 
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                         Test set:	30_protein_test.csv	                         
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
 Acc level 1:	96.95	
Acc level 2:	91.6	
Acc level 3:	87.79	
Acc level 4:	81.68	 
--------------------------------------------------------------------------------
----------------------------

In [26]:
compute_accuracy_baseline1(results['predicted_ecs'].values, results['true_ecs'].values)

--------------------------------------------------------------------------------
Acc level 1:	96.15	
Acc level 2:	92.86	
Acc level 3:	90.11	
Acc level 4:	84.07	 
--------------------------------------------------------------------------------


(0.9615384615384616,
 0.9285714285714286,
 0.9010989010989011,
 0.8406593406593407)